# Segmenting and Clustering Neighborhoods in Toronto : Part 3 
## Explore and cluster the neighborhoods in Toronto

----

#### Importing python notebooks for part2 of the problem to share variables and namespace

In [1]:
%run Geographical_Coordinate_DataFrame.ipynb

In [2]:
# Import Libraries
import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Creating Dataframe for Analysis
For clustering exercise, we will use neighboods in boroughs with names containing Toronto

In [3]:
# check which borough names contain 'Toronto'
borough_toronto_msk =[]
for x in df['Borough']:
    borough_toronto_msk.append('Toronto' in x)

In [4]:
# using the boolean mask above, create the new dataframe to be used for clustering
newdf = df[borough_toronto_msk]
toronto_data = newdf
toronto_data.reset_index(drop=True, inplace=True)

#check the datafram
toronto_data.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


Create another dataframe where names of neighborhood are not joint (as needed in part2 of assignment), this will be used to join data frame with venues, which will be used for plotting maps

In [5]:
toronto_geolist = pd.DataFrame(columns={'Postcode','Borough','Neighborhood','Latitude','Longitude'})
for index,rows in toronto_data.iterrows():
    postcode, borough, neighbor, lat, lng = rows
    for neigh in neighbor.split(","):
        toronto_geolist=toronto_geolist.append({'Postcode':postcode,'Borough':borough,'Neighborhood':neigh,'Latitude':lat,'Longitude':lng}, ignore_index=True)

toronto_geolist=toronto_geolist[['Postcode','Borough','Neighborhood','Latitude','Longitude']]
toronto_geolist.reset_index(drop=True, inplace=True)
# check the dataframe 
toronto_geolist.head(15)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M4E,East Toronto,The Beaches,43.676357,-79.293031
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M6G,Downtown Toronto,Christie,43.669542,-79.422564
9,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568


## Visualize the neighborhood of toronto using Folium map
Get the coordinates of Toronto city using geopy's nominatim function

In [6]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# print the coordinates of Toronto

In [7]:

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude+0.01, longitude], zoom_start=12)

# add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(newdf['Latitude'], newdf['Longitude'], newdf['Borough'], newdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
# plot the map
map_toronto

## Fetch venues in each neighborhood using Fourspace API and create data to be used in k-means clustering
To hide login credentials of fourspace API, another notebook is imported below. It declares client ID and secret variables, along with version, radius and limit of query

In [9]:
%run fourspace_login.ipynb

Create function function getNearbyVenues() to fetch venues using Foursquare API in json, followed json normalization and extractio through indexing of lists and transfer into dataframe

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name1, lat, lng in zip(names, latitudes, longitudes):
        for name in name1.split(","):
            
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)
            
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results]
            )
                
# create new data frame and transfer values from list in columns
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    
# return the dataframe with venues for neighborhood 
    return(nearby_venues)

create new dataframe with venues for each neighborhood using function created above

In [11]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [12]:
# Show new dataframe with venues
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Show number of unique venue categories for all neighborhoods

In [13]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


Create one hot encoding for venue categores for each neighborhood

In [14]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(columns=['Neighborhood'], inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

Create grouped by venue dataframe for each neighborhood

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.01,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,0.010000,0.01,0.0,0.03,0.000000,0.000000,0.0,0.000000,0.030000,0.0,0.040000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.01,0.000000,0.01,0.020000,0.000000,0.0,0.01,0.03,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.010000,0.060000,0.0,0.0,0.0,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.0,0.000000,0.0,0.01,0.0,0.01,0.01,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.00000

create the new dataframe and display the top 10 venues for each neighborhood.

In [16]:
# create new function which returns top 10 venues from grouped row
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

# declare variables
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# arrange columns of new dataframe
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#view the new dataframe
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Burger Joint,Gym,Bakery,Sushi Restaurant,Hotel
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Steakhouse,Bakery,Café,Farmers Market,Seafood Restaurant,Italian Restaurant,Pub
3,Brockton,Coffee Shop,Breakfast Spot,Café,Convenience Store,Burrito Place,Stadium,Caribbean Restaurant,Bar,Restaurant,Italian Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Restaurant


## Create k-means clusters using top 10 venue in each neighborhood

In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
       3, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 3, 2, 0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# use the dataframe created earlier which contains coordinates of neighborhood
toronto_merged = toronto_geolist

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# show the new dataframe
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Electronics Store
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Electronics Store
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Lingerie Store,Bar,Japanese Restaurant,Tea Room,Italian Restaurant
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Lingerie Store,Bar,Japanese Restaurant,Tea Room,Italian Restaurant
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Breakfast Spot,Bakery,Hotel,Gastropub,Italian Restaurant,Park,Pizza Place


## Create map to visualize the clustering results

In [19]:
# create base map
map_clusters = folium.Map(location=[latitude+0.01, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_clusters)

#print the map       
map_clusters

## Examine Clusters
Tabulate each cluster of neighborhoods with venues to visualize any systematic difference among the clusters

#### Cluster 0

In [20]:
c0=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
c0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,0,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Electronics Store
1,Regent Park,0,Coffee Shop,Café,Park,Pub,Bakery,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Electronics Store
2,Ryerson,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Lingerie Store,Bar,Japanese Restaurant,Tea Room,Italian Restaurant
3,Garden District,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Lingerie Store,Bar,Japanese Restaurant,Tea Room,Italian Restaurant
4,St. James Town,0,Coffee Shop,Restaurant,Café,Breakfast Spot,Bakery,Hotel,Gastropub,Italian Restaurant,Park,Pizza Place
5,The Beaches,0,Health Food Store,Coffee Shop,Pub,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
6,Berczy Park,0,Coffee Shop,Restaurant,Cocktail Bar,Steakhouse,Bakery,Café,Farmers Market,Seafood Restaurant,Italian Restaurant,Pub
7,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Burger Joint,Bar,Bubble Tea Shop,Thai Restaurant,Sushi Restaurant,Salad Place,Chinese Restaurant
8,Christie,0,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Diner,Restaurant,Nightclub,Baby Store,Convenience Store
9,Adelaide,0,Coffee Shop,Bar,Steakhouse,Café,Thai Restaurant,Burger Joint,Gym,Bakery,Sushi Restaurant,Hotel


In [21]:
c0[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Coffee Shop,Coffee Shop,Café


#### Cluster 1

In [22]:
c1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
c1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,CN Tower,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
60,Bathurst Quay,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
61,Island airport,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
62,Harbourfront West,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
63,King and Spadina,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
64,Railway Lands,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden
65,South Niagara,1,Airport Lounge,Airport Service,Airport Terminal,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Boutique,Boat or Ferry,Sculpture Garden


In [23]:
# Find mode (most offen occuring) venue in 1st, 2nd and 3rd most common venues
c1[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Airport Lounge,Airport Service,Airport Terminal


#### Cluster 2

In [24]:
c2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
c2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Roselawn,2,Home Service,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [25]:
# Find mode (most offen occuring) venue in 1st, 2nd and 3rd most common venues
c2[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Home Service,Garden,Yoga Studio


#### Cluster 3

In [26]:
c3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
c3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Forest Hill North,3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
35,Forest Hill West,3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
66,Rosedale,3,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [27]:
# Find mode (most offen occuring) venue in 1st, 2nd and 3rd most common venues
c3[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Park,Trail,Sushi Restaurant


#### Cluster 4

In [28]:
c4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
c4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Moore Park,4,Playground,Tennis Court,Concert Hall,Convenience Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
50,Summerhill East,4,Playground,Tennis Court,Concert Hall,Convenience Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [29]:
# Find mode (most offen occuring) venue in 1st, 2nd and 3rd most common venues
c4[['1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue']].mode()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Playground,Tennis Court,Concert Hall


### Examination of common venues among clusters show the basis of clustering
#### Example see 1st and 2nd venues for each cluster below :
#### Cluster 0 ~ Coffee Shop / Coffee Shop / Cafe
#### Cluster 1 ~ Air Port Longe / Airport Service
#### Cluster 2 ~ Home Service / Garden
#### Cluster 3 ~ Park / Trail
#### Cluster 4 ~ Playground / Tennis Court